- more decided on looking at larger picture and finding places where you can win with 2 consecutive moves - how to create score/algorithm to make that
- does score1 actually make a difference, maybe just score2 with no passed on parameters
- make syntax of everyting clean 

In [7]:
import numpy as np
import random

In [8]:
def get_lines(grid: np.ndarray):
#     returns a list of lines (that have a length greater than 4) from the grid - rows, columns, both diagonals
    return [[list(x) for x in grid], 
            [list(grid[:,c]) for c in range(7)], 
            [list(np.diag(grid, x)) for x in range(-2, 4)], 
            [list(np.diag(np.fliplr(grid), x)) for x in range(-2, 4)]]

In [9]:
# a Grid Reference class to locate lines from a specific coordinate

class GridRef:
    def __init__(self):
        self.number_grid = np.arange(42).reshape(6,7)
        self.lines_dict = {n: [None] * 4 for n in range(42)}
        self.apply_lines_to_lines_dict()
        
    def apply_lines_to_lines_dict(self):
        groups = get_lines(self.number_grid)
        for i in range(42):
            for r in range(7):
                for ind in range(4):
                    if r == 6:
                        if i in groups[1][r]:
                            self.lines_dict[i][1] = r
                    elif ind < 2:
                        if i in groups[ind][r]:
                            self.lines_dict[i][ind] = r
                    else:
                        if i in groups[ind][r]:
                            self.lines_dict[i][ind] = r - 2
                            
grid_reference = GridRef()
# grid_reference.get_lines_from_position(3,2)

In [31]:
class Grid:
    def __init__(self): 
        self.player_1 = 'X'
        self.player_2 = 'O'
        self.current_player = self.player_1
        self.opposition = self.player_2
        self.grid = np.full((6,7), None)
        self.space = {n: 5 for n in range(7)}
        self.children = {}
        self.level = 0
        self.score = 0
        self.winner = False

    def make_move(self, player, column):
#     inputs the player counter into the next available space in the column, then changes the current player
        row = self.space[column]
        self.grid[row, column] = player
        self.space[column] -= 1
        self.current_player, self.opposition = self.opposition, self.current_player
        
    def winner_search_original(self, lines):
#     searches the lines parameter for a winner (four in a row of anything except None)
        for line in lines:
            count = 1
            for i in range(1, len(line)):
                if line[i] == line[i - 1]:
                    count += 1
                else:
                    count = 1
                if count == 4:
                    if not line[i]:
                        continue
                    return line[i]
        return False
    
    def winner_search(self, lines):
#         new winner search with numpy technique. on runnig seems slower than original
        for line in lines:
            sub_lines = [line[x - 4: x] for x in range(4, len(line) + 1)]
            for sub in sub_lines:
                if sum(sub == self.player_1) == 4:
                    return self.player_1
                elif sum(sub == self.player_2) == 4:
                    return self.player_2
        return False

    def get_winner(self):
#     searches all lines in a grid for the winner
        return self.winner_search(get_lines(self.grid)[::-1])


    def get_winner2(self, row, column):
#     searches just the lines that intersect at coordinates grid[row,column]
        return self.winner_search(self.get_lines_from_position(row, column))
    

    def score_search(self, lines):
#     score calculated by opportunities to make four in a row of one player minus opportunities for the other player
        scores = [0, 0]
        for lsts in lines:
            for line in lsts:
                sub_lines = [line[x - 4: x] for x in range(4, len(line) + 1)]
                for i in range(2):
                    for sub in sub_lines:
                        if not (([self.player_1, self.player_2][i - 1] in sub) or (set(sub) == {None})):
                            scores[i] += 1
                            break
        return scores[0] - scores[1]

    def get_score(self):
        return self.score_search(get_lines(self.grid))
    
    def get_score2(self, row, column):
        return self.score_search([self.get_lines_from_position(row, column)])
    
    def make_children(self, depth_limit):
#         the children are the possible moves from the current grid. The depth is how many times this iterates.
        if (self.level == depth_limit) or self.winner:
            return
        for col in range(7):
            if self.space[col] >= 0:
                self.children[col] = GridChild(self.grid.copy(), 
                                               self.space.copy(), 
                                               self, 
                                               self.level, 
                                               self.current_player, 
                                               self.opposition, 
                                               depth_limit, 
                                               self.space[col], 
                                               col, 
                                               self.score)
                
    def get_children_from_level(self, lev):
        children = [self]
        next_children = []
        while children[0].level < lev:
            for child in children:
                next_children += child.children.values()
            children, next_children = next_children, []
        return children

    def get_lines_from_position(self, x, y):
#         gets the lines from coordinates[x,y]
        l1, l2, l3, l4 = grid_reference.lines_dict[grid_reference.number_grid[x,y]]
        lines = []
        lines.append(self.grid[l1,:])
        if isinstance(l2, int):
            lines.append(self.grid[:, l2])
        if isinstance(l3, int):
            lines.append(np.diag(self.grid, l3))
        if isinstance(l4, int):
            lines.append(np.diag(np.fliplr(self.grid), l4))
        return lines
                        
    def print_grid(self):
        new_grid = self.grid.copy()
        new_grid[new_grid == None] = '_'
        print()
        print(new_grid)

        
        

In [32]:
g6 = Grid()

# g6.make_children(6)

# for x in g6.get_children_from_level(4):
#     print(x)

In [33]:
class GridChild(Grid):
    def __init__(self, grid, space, parent, level, current_player, opposition, depth_limit, row, col, score):
        self.player_1 = 'X'
        self.player_2 = 'O'
        self.current_player = current_player
        self.opposition = opposition
        self.grid = grid
        self.space = space
        self.parent = parent
        self.children = {}
        self.level = level + 1
        # self.carry = 0
        self.last_move = [row, col]
        self.make_move(self.current_player, col)
        self.winner = self.get_winner2(row, col)
        self.score = self.get_score2(row, col)
        # self.score2 = score + self.get_score2(row, col) + self.carry
        self.make_children(depth_limit)
        
    def make_move(self, player, column):
        # self.carry = - self.get_score2(self.space[column], column)
        super().make_move(player, column)
        
# score seems to be faster than score2, but from make_children the score function definitely takes up the most time


In [34]:
class TestGrid(Grid):
    
    def make_random_grid(self):
        pool = ['X', 'O', None]
        grid = []
        for x in range(6):
            grid.append(random.choices(pool, k = 7))
        return np.array(grid)
    
    def test_play(self):
        self.grid = self.make_random_grid()
        print(self.grid)
        return self.get_winner()
    
    def test_random_till_winner(self):
        order = []
        while True:
            players = ['X', 'O']
            for p in players:
                while True:
                    col = random.randint(0,6)
                    if self.space[col] >= 0:
                        break
                row = self.space[col]
                self.make_move(p, col)
                winner = self.get_winner2(row, col)
                order.append(p + str(col))
                if winner:
                    self.print_grid(self.grid)
                    print()
                    return winner, self.get_score()
    
    def two_player_game(self):
        print('playing game')
        while True:
            computer_moved = False
            if set(self.space.values()) == {-1}:
                print('No Winner')
                return
            while True:
                turn = input('input 0 - 6 for the column')
                if turn.isdigit() and 0 <= int(turn) < 7 and self.space[int(turn)] >= 0:
                    break
            row = self.space[int(turn)]
            self.make_move(self.player_1, int(turn))
            self.print_grid()
            if self.get_winner2(row, int(turn)):
                print('You have won')
                return
            
#             computer turn

            self.make_children(4)
            for child in self.get_children_from_level(1):
                if child.winner:
                    self.make_move(self.player_2, child.last_move[1])
                    self.print_grid()
                    print('computer wins')
                    return
            parents = {}
            for grandchild in self.get_children_from_level(2):
                if grandchild.winner:
                    parents[grandchild.parent.last_move[1]] = grandchild.parent
            if parents:
                moves = {0,1,2,3,4,5,6}.difference(set(parents.keys()))
                moves = list(filter(lambda x: self.space[x] >= 0, list(moves)))
                if moves:
                    new_moves = [self.children[x] for x in moves]
                    new_moves = sorted(new_moves, key = lambda x: x.score)
#                     print([m.last_move[1] for m in new_moves])
                    move = new_moves[0].last_move[1]
                    self.make_move(self.player_2, move)
                    computer_moved = True
            level_3_grandparents = {n: set() for n in range(7)}
            for level_3 in self.get_children_from_level(3):
                if level_3.winner:
                    level_3_grandparents[level_3.parent.parent.last_move[1]].add(level_3.parent.last_move[1])
            available_len = len(list(filter(lambda x:self.space[x] >= 0, [0,1,2,3,4,5,6])))
            for key in level_3_grandparents:
                if len(level_3_grandparents[key]) == available_len:
                    self.make_move(self.player_2, key)
                    computer_moved = True
                    print('lev 3 moved', key)
            if not computer_moved:
                level_4_grandparents = {}
                for level_1 in self.children.values():
                    m1 = level_1.last_move[1]
                    available_len = len(list(filter(lambda x:self.space[x] >= 0, [0,1,2,3,4,5,6])))
                    l1g = {n: set() for n in range(7)}
                    for level_2 in level_1.children.values():
                        for level_3 in level_2.children.values():
                            for level_4 in level_3.children.values():
                                if level_4.winner:
                                    l1g[level_4.parent.parent.last_move[1]].add(level_4.parent.last_move[1])
                    level_4_grandparents[m1] = any([len(l1g[key]) == available_len for key in l1g.keys()])
                print(level_4_grandparents)
                if any(level_4_grandparents.values()):
                    moves = list(filter(lambda x: not level_4_grandparents[x], level_4_grandparents.keys()))
                    if moves:
                        print('bad news in a few turns', moves)
                        new_moves = sorted([self.children[c] for c in moves], key = lambda x: x.score)
                        self.make_move(self.player_2, new_moves[0].last_move[1])
                        computer_moved = True

            if not computer_moved:
                moves = list(filter(lambda x:self.space[x] >= 0, [0,1,2,3,4,5,6]))
                new_moves = [self.children[x] for x in moves]
                new_moves = sorted(new_moves, key = lambda x: x.score)
                print([m.last_move[1] for m in new_moves])
                move = random.choice(new_moves[:3]).last_move[1]
                self.make_move(self.player_2, move)
                computer_moved = True
            if computer_moved:
                self.print_grid()
                self.children = {}   
        return 

In [2]:
# flatten massive list thing into a list of lists

class ListFlatten:
    def __init__(self, group):
        self.result = []
        
    def flatten(self):
        self.backtrack(group)
        return self.result
        
    def backtrack(self, group):
        if type(group[0]) != list:
            self.result.append(group)
        else:
            for item in group:
                self.backtrack(item)
        

In [204]:
import random
pool = [0,1,2,0]
dict = {0: 0, 1: 1, 2: -1}
lsts = [random.choices(pool, k= 7) for x in range(5)]

def score1(lsts):
    # score function that says how many lines are active and can be filled to win
    score = 0
    for lst in lsts:
        sub_list = [lst[i: i + 4] for i in range(len(lst) - 3)]
        lst_score = set()
        for sub in sub_list:
            four_score = set()
            for n in sub:
                four_score.add(dict[n])
            lst_score.add(sum(four_score))
        score += sum(lst_score)
    return score

In [340]:
def score2(lsts):
    # score function that takes how many single pieces are active (able to make a winning four in the future)
    # loops through counting pieces that are currently active, then adding their values if they make up a length
    # greater than 4 without the other token in the set. 
    score = 0
    for lst in lsts:
        sett, count, length, zeros = set(), 0, 0, 0
        for n in lst:
            sett.add(dict[n])
            if sett.intersection({1,-1}) == {1, -1}:
                if length >= 4:
                    score += count * sum(sett.difference({dict[n]}))
                sett = {dict[n]}
                count = 1 
                length = 1 + zeros
                zeros = 0
                continue
            if n != 0:
                zeros = 0
                count += 1
            else: 
                zeros += 1
            length += 1
        if length >= 4:
            score += count * sum(sett)
    return score    

In [1]:
def score3(lsts):
    # same function as score2 but different implementation. seems to be a bit slower.
    score = 0
    for lst in lsts:
        accounted_for = set()
        sub_lsts = [np.array(lst[i: i + 4]) for i in range(len(lst) - 3)]
        count = 0
        for l in sub_lsts:
            if set(l).intersection({1,2}) == {1,2}:
                count += 1
                continue
            active = np.nonzero(l)[0]
            active = active + count
            new = set(active).difference(accounted_for)
            if new:
                value = dict[lst[list(new)[0]]]
                score += value * len(new)
                accounted_for.update(new)
            count += 1
    return score

In [3]:
import math

math.inf < 100000000000000

False

In [4]:
p = math.inf
m = -math.inf

In [7]:
abs(m) == abs(p)

True

In [1]:
s = 'sid'

In [12]:
import numpy as np
s = np.array([1,2,3,4,0,2,3, 0])

In [13]:
np.nonzero(s == 0)[0][0]

4

In [27]:
if np.where(s == 3)[0].size:
    print('yes')

yes


In [ ]:
[np.nonzero(s == 0)[0][0] for col in range(len(s)) if np.nonzero(s == 0)[0].size else 10]